In [1]:
import os

In [2]:
%pwd

'/Users/sarutsunpawatr/Datath Online courses/E2Eproject/MLflowdemp/MLflow/End-to-End-project-classification-using-MLflow-DVC-on-Aws/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sarutsunpawatr/Datath Online courses/E2Eproject/MLflowdemp/MLflow/End-to-End-project-classification-using-MLflow-DVC-on-Aws'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]= "https://dagshub.com/parklhbd45/End-to-End-project-classification-using-MLflow-DVC-on-Aws.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]= "parklhbd45"
os.environ["MLFLOW_TRACKING_PASSWORD"]= "9a9fe39d51984c009d127024ac1d96fbe0ddc4a9"

In [6]:
import os

# ตรวจสอบว่าตัวแปรเหล่านี้ถูกตั้งค่าแล้ว
print(os.environ.get('MLFLOW_TRACKING_USERNAME'))
print(os.environ.get('MLFLOW_TRACKING_PASSWORD'))
# หรือ
print(os.environ.get('MLFLOW_TRACKING_TOKEN'))

parklhbd45
9a9fe39d51984c009d127024ac1d96fbe0ddc4a9
None


In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/parklhbd45/End-to-End-project-classification-using-MLflow-DVC-on-Aws.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
from mlflow.exceptions import MlflowException
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from tensorflow import keras


In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):

        try:
            mlflow.set_tracking_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # ใช้ชื่อ experiment แบบ hardcode หรือใช้ชื่อโปรเจกต์ของคุณ
            experiment_name = "ChestClassifier_Experiment"

            # Check if the experiment exists, if not create it
            experiment = mlflow.get_experiment_by_name(experiment_name)
            if experiment is None:
                experiment_id = mlflow.create_experiment(experiment_name)
            else:
                experiment_id = experiment.experiment_id

            # สร้างชื่อ run โดยใช้เวลาปัจจุบัน
            from datetime import datetime
            run_name = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

            with mlflow.start_run(experiment_id=experiment_id, run_name=run_name):
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

                # Model registry does not work with file store
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")

                print("Model logged in MLflow successfully")

        except MlflowException as e:
            print(f"Error logging to MLflow: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-20 13:44:23,706: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-20 13:44:23,707: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-20 13:44:23,708: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 8s 1s/step - loss: 9.2682 - accuracy: 0.4314
[2024-10-20 13:44:31,512: INFO: common: json file saved at: scores.json]


2024/10/20 13:44:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/09/541wm2pd45g8knrnkz24_3y80000gn/T/tmpl552zzkb/model/data/model/assets
[2024-10-20 13:44:34,118: INFO: builder_impl: Assets written to: /var/folders/09/541wm2pd45g8knrnkz24_3y80000gn/T/tmpl552zzkb/model/data/model/assets]


/Users/sarutsunpawatr/anaconda3/envs/chestclassifier/lib/python3.9/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/20 13:44:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.


Model logged in MLflow successfully


In [17]:
eval_config.path_of_model

'artifacts/training/model.h5'

In [15]:
help(eval_config)

Help on EvaluationConfig in module __main__ object:

class EvaluationConfig(builtins.object)
 |  EvaluationConfig(path_of_model: pathlib.Path, training_data: pathlib.Path, all_params: dict, mlflow_uri: str, params_image_size: list, params_batch_size: int) -> None
 |  
 |  EvaluationConfig(path_of_model: pathlib.Path, training_data: pathlib.Path, all_params: dict, mlflow_uri: str, params_image_size: list, params_batch_size: int)
 |  
 |  Methods defined here:
 |  
 |  __delattr__(self, name)
 |  
 |  __eq__(self, other)
 |  
 |  __hash__(self)
 |  
 |  __init__(self, path_of_model: pathlib.Path, training_data: pathlib.Path, all_params: dict, mlflow_uri: str, params_image_size: list, params_batch_size: int) -> None
 |  
 |  __repr__(self)
 |  
 |  __setattr__(self, name, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |  